<a href="https://colab.research.google.com/github/TSHP/DL4SC_Neural_ODEs/blob/main/ToyExampleTobia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading in the MNIST DATASET

In [124]:
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import torch.nn as nn
from torch.nn import ReLU
from torch import optim
import matplotlib.pyplot as plt
from torch.nn.functional import one_hot

In [125]:

mnistTrainSet = torchvision.datasets.MNIST(root='./data', train=True,
                                    download=True, transform=transforms.ToTensor()) # might need to normalize
mnistTrainLoader = torch.utils.data.DataLoader(mnistTrainSet, batch_size=64,
                                      shuffle=True)

Constructing a 1 Layer MLP

In [126]:
class MLP(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(MLP, self).__init__()
        self.linear = nn.Linear(in_dim, out_dim)
        self.Relu = ReLU()
    def forward(self, x):
        out = self.linear(x)
        out = self.Relu(out)
        return out

In [136]:
batch_size = 64
in_dim, num_class = 28*28, 10

l_r = 0.0001
w_decay = 1e-4
model = MLP(in_dim, num_class)
optimizer = optim.Adam(model.parameters(), lr=l_r, weight_decay = w_decay)
criterion = nn.CrossEntropyLoss()

Train with 10 class probabilities

In [137]:
total_steps = len(mnistTrainLoader)
epochs = 10
for cur_epoch in range(epochs):
  for x, (images, labels) in enumerate(mnistTrainLoader):
    images = images.reshape(-1, 28*28)

    out = model(images)

    losses = criterion(out, labels)

    optimizer.zero_grad()
    losses.backward()
    optimizer.step()
    if (x+1) % 300 == 0:
      print (f'Epoch[{cur_epoch+1}/{epochs}], Step[{x+1}/{total_steps}], Losses: {losses.item():.4f}')
print(losses.item())

Epoch[1/10], Step[300/938], Losses: 1.7000
Epoch[1/10], Step[600/938], Losses: 1.2934
Epoch[1/10], Step[900/938], Losses: 1.0228
Epoch[2/10], Step[300/938], Losses: 1.0180
Epoch[2/10], Step[600/938], Losses: 0.8027
Epoch[2/10], Step[900/938], Losses: 0.8379
Epoch[3/10], Step[300/938], Losses: 0.7958
Epoch[3/10], Step[600/938], Losses: 0.6384
Epoch[3/10], Step[900/938], Losses: 0.7302
Epoch[4/10], Step[300/938], Losses: 0.6881
Epoch[4/10], Step[600/938], Losses: 0.7572
Epoch[4/10], Step[900/938], Losses: 0.7687
Epoch[5/10], Step[300/938], Losses: 0.8769
Epoch[5/10], Step[600/938], Losses: 0.4406
Epoch[5/10], Step[900/938], Losses: 0.6710
Epoch[6/10], Step[300/938], Losses: 0.5670
Epoch[6/10], Step[600/938], Losses: 0.4637
Epoch[6/10], Step[900/938], Losses: 0.4917
Epoch[7/10], Step[300/938], Losses: 0.7499
Epoch[7/10], Step[600/938], Losses: 0.5715
Epoch[7/10], Step[900/938], Losses: 0.6563
Epoch[8/10], Step[300/938], Losses: 0.6835
Epoch[8/10], Step[600/938], Losses: 0.5051
Epoch[8/10]

Dataloader Understanding

In [ ]:
plt.imshow(mnistTrainSet.data[0])
plt.show()

In [ ]:
for idx, batch in enumerate(mnistTrainLoader):
  print('Batch index: ', idx)
  print('Batch size: ', batch[0].size())
  print('Batch label: ', batch[1])
  break

In [ ]:
for idx, (data, target) in enumerate(mnistTrainLoader):
    print(data[0].shape)
    print(len(data))
    print(len(target))
    print(target[0])
    break